TASK-1

In [1]:
#pyspark setup

!apt-get update
# Install Java 8 (required by Spark)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Install Spark
!pip install pyspark

# setup environment variables
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# Initialize Spark session
spark = SparkSession.builder.appName("Retail Sales Analytics").getOrCreate()

# Load datasets
sales_df = spark.read.csv("/workspaces/Project-2-Retail-Sales-Analytics-and-Real-Time-Demand-Forecasting/task1/Online Retail - Online Retail (1).csv", header=True, inferSchema= True) # Corrected the filename
reviews_df = spark.read.csv("/workspaces/Project-2-Retail-Sales-Analytics-and-Real-Time-Demand-Forecasting/task1/reviews.csv", header=True, inferSchema=True)

# Explore data
sales_df.show(10)
reviews_df.show(10)

# Check schema
sales_df.printSchema()
reviews_df.printSchema()


24/12/06 03:56:02 WARN Utils: Your hostname, codespaces-3e6bbc resolves to a loopback address: 127.0.0.1; using 10.0.2.121 instead (on interface eth0)
24/12/06 03:56:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 03:56:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/10 8:26|     4.25|     17850|United

24/12/06 03:56:14 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
from pyspark.sql.functions import col

print("Missing Values in Sales Data:")
sales_df.select([col(column).isNull().alias(column) for column in sales_df.columns]).show()

print("Missing Values in Reviews Data:")
reviews_df.select([col(column).isNull().alias(column) for column in reviews_df.columns]).show()

Missing Values in Sales Data:
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|    

In [4]:
sales_df.select([(col(c).isNull().cast("int")).alias(c) for c in sales_df.columns]).summary().show()
reviews_df.select([(col(c).isNull().cast("int")).alias(c) for c in reviews_df.columns]).summary().show()

24/12/06 03:56:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+---------+--------------------+--------+-----------+---------+------------------+-------+
|summary|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|        CustomerID|Country|
+-------+---------+---------+--------------------+--------+-----------+---------+------------------+-------+
|  count|   541909|   541909|              541909|  541909|     541909|   541909|            541909| 541909|
|   mean|      0.0|      0.0|0.002683107311375157|     0.0|        0.0|      0.0| 0.249266943342886|    0.0|
| stddev|      0.0|      0.0|0.051729229498118534|     0.0|        0.0|      0.0|0.4325890424198415|    0.0|
|    min|        0|        0|                   0|       0|          0|        0|                 0|      0|
|    25%|        0|        0|                   0|       0|          0|        0|                 0|      0|
|    50%|        0|        0|                   0|       0|          0|        0|                 0|      0|
|    75%|        0|

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("Data Cleansing - Simplified Reviews").getOrCreate()

# Load the review dataset
reviews_data_path = "/workspaces/Project-2-Retail-Sales-Analytics-and-Real-Time-Demand-Forecasting/task1/reviews.csv"  # Replace with the actual file path
reviews_df = spark.read.csv(reviews_data_path, header=True, inferSchema=True)

# Inspect data
print("Initial dataset:")
reviews_df.show(10)
reviews_df.printSchema()

# Handle missing values
print("Handling missing values...")
reviews_df = reviews_df.dropna(subset=["Review", "Sentiment"])

# Debugging step: Check dataset after dropping missing values
print("Dataset after dropping missing values:")
reviews_df.show(10)

# Remove rows with empty or whitespace-only reviewText
print("Filtering non-meaningful text...")
reviews_df = reviews_df.filter(col("Review").rlike(r"\S+"))

# Debugging step: Check dataset after filtering non-meaningful text
print("Dataset after filtering non-meaningful text:")
reviews_df.show(10)

# Validate sentiment values (should be "positive" or "negative")
print("Validating sentiment values...")
valid_sentiments = ["positive", "negative"]
reviews_df = reviews_df.filter(col("Sentiment").isin(valid_sentiments))

# Debugging step: Check dataset after validating sentiment
print("Dataset after validating sentiment:")
reviews_df.show(10)

# Drop duplicates
print("Dropping duplicates...")
reviews_df = reviews_df.dropDuplicates()

# Final dataset preview
print("Final cleaned dataset:")
reviews_df.show(10)

# Save cleaned data for further use
output_path = "cleaned_reviews_data.csv"  # Replace with your desired output path
reviews_df.write.csv(output_path, header=True, mode="overwrite")
print(f"Cleaned dataset saved to {output_path}")


24/12/06 03:56:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Initial dataset:
+--------------------+---------+
|              Review|Sentiment|
+--------------------+---------+
|This product exce...| Positive|
|The product was d...|  Neutral|
|I had a terrible ...| Negative|
|It's an okay prod...|  Neutral|
|Disappointed with...| Negative|
|Avoid this compan...| Negative|
|I had a terrible ...| Negative|
|Avoid this compan...| Negative|
|This product exce...| Positive|
|This product is o...| Positive|
+--------------------+---------+
only showing top 10 rows

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)

Handling missing values...
Dataset after dropping missing values:
+--------------------+---------+
|              Review|Sentiment|
+--------------------+---------+
|This product exce...| Positive|
|The product was d...|  Neutral|
|I had a terrible ...| Negative|
|It's an okay prod...|  Neutral|
|Disappointed with...| Negative|
|Avoid this compan...| Negative|
|I had a terrible ...| Negative|
|Avoid this co

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col

spark = SparkSession.builder.appName("Retail Sales Analytics").getOrCreate()

# Set legacy time parser policy to handle the problematic date format
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Load datasets
sales_df = spark.read.csv("/workspaces/Project-2-Retail-Sales-Analytics-and-Real-Time-Demand-Forecasting/task1/Online Retail - Online Retail (1).csv", header=True, inferSchema= True) # Corrected the filename


# Inspect data
sales_df.show(10)
sales_df.printSchema()

# Handle missing values
# Drop rows with null CustomerID or Description, as they are critical fields
sales_df = sales_df.dropna(subset=["StockCode", "Description"])

# Drop duplicates
sales_df = sales_df.dropDuplicates()

# Parse InvoiceDate to a proper date format
# Use the correct format based on your data (e.g., "dd/MM/yy HH:mm")
sales_df = sales_df.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "dd/MM/yy HH:mm"))

# Filter out invalid data (e.g., Quantity or UnitPrice less than or equal to 0)
sales_df = sales_df.filter((col("Quantity") > 0) & (col("UnitPrice") > 0))

# Final schema and data preview
sales_df.printSchema()
sales_df.show(10)


24/12/06 03:56:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/10 8:26|     4.25|     17850|United

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536375|   84406B|CREAM CUPID HEART...|       8| 2010-01-12|     2.75|     17850|United Kingdom|
|   536389|   85014A|BLACK/BLUE POLKAD...|       3| 2010-01-12|     5.95|     12431|     Australia|
|   536408|    22492|MINI PAINT SET VI...|      36| 2010-01-12|     0.65|     14307|United Kingdom|
|   536425|    22829|SWEETHEART WIRE W...|       2| 2010-01-12|     9.95|     13758|United Kingdom|
|   536437|    22189|CREAM HEART CARD ...|      72| 2010-01-12|     3.39|     13694|United Kingdom|
|   536528|   85114C|RED ENCHANTED FOR...|       1| 2010-01-12|     1.65|     15525|United Kingdom|
|   536532|    22555|PLASTERS IN TIN S...|      36| 2010-01-12|     1.65|     12433|        Norway|


In [8]:
# Save cleaned datasets for downstream tasks
sales_df.write.csv("cleaned_sales_data.csv", header=True, mode="overwrite")
reviews_df.write.csv("cleaned_reviews_data.csv", header=True, mode="overwrite")